In [1]:
from bs4 import BeautifulSoup
import lxml
import pandas as pd

In [2]:
with open('List of postal codes of Canada_ M - Wikipedia.htm') as html_file:
    soup = BeautifulSoup(html_file, 'lxml')

In [56]:
# Get HTML content and insert into structured list
table = soup.find('table', class_='wikitable sortable jquery-tablesorter')

rows = []
for row in table.find_all('tr'):
    rows.append(row.text.split('\n'))

# Convert to dataframe and add headers
headers = ['PostalCode', 'Borough', 'Neighborhood']

df = pd.DataFrame(rows[1:])
df.drop([0,4], axis=1, inplace=True)
df.columns = headers

# Drop boroughs that are not assigned
df = df[df.Borough != 'Not assigned']

pd.set_option('display.max_rows', 50)
df.head(20)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [57]:
# Group Neighborhoods that have the same PostalCode and Borough together
df_clean = df.groupby(['PostalCode', 'Borough']).Neighborhood.apply(list).reset_index()
df_clean['Neighborhood'] = df_clean['Neighborhood'].apply(', '.join)

# Replace 'Not assigned' Neighborhood values with Borough value
df_clean.loc[df_clean['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df_clean['Borough']

df_clean.head(20)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [22]:
df_clean.shape

(103, 3)

In [59]:
# Get coordinates
df_pcodes = pd.read_csv('Geospatial_Coordinates.csv')
df_pcodes.rename(index=str, columns={'Postal Code':'PostalCode'}, inplace=True)

# Merge with df_clean dataframe
df_final = df_clean.merge(df_pcodes, how='inner', on=['PostalCode'])

df_final.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
